In [ ]:
# from pyspark import SparkContext

# sc = SparkContext("local", "This First APP ~~")

In [21]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## **Pandas**

In [14]:
from pyspark import pandas as ps
import pandas as pd

In [18]:
# From csv file
df = ps.read_csv('data/03.csv')
df

/usr/local/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,A,B,C
0,1,2.0,3.0
1,4,5.0,6.0
2,7,8.0,9.0


In [19]:
# From dict
df = ps.DataFrame(
    {
        'A': list('123'),
        'B': list('234'),
        'C': list('567')
    }
)

df

/usr/local/lib/python3.10/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


,A,B,C
0,1,2,5
1,2,3,6
2,3,4,7


In [20]:
# From DataFrame
df = ps.DataFrame(
    pd.DataFrame(
        {
            'A': list('123'),
            'B': list('234'),
            'C': list('567')
        }        
    )
)
df

/usr/local/lib/python3.10/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


,A,B,C
0,1,2,5
1,2,3,6
2,3,4,7


In [ ]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

In [ ]:
df.show()